Here I tried to build a inductive dataset from the single traces of death star dataset, however it is not possible since the traces are so small that need to be all part of the train set in order to be a inductive set: each relation is present just one time in the dataset, therefore it is not possible to create a dataset. Also trying to put all the traces together doesn't work.

In [1]:
import json
import networkx as nx
import networkit as nk
import matplotlib.pyplot as plt
import math
import numpy as np
from numpy.linalg import inv
import pandas as pd
import random


seed = 1234

In [2]:
def create_triplets(path):
  
    # Opening JSON file
    f = open(path)

    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    # Closing file
    f.close()
    content = data['data']
    first = content[0]
    second = first['spans']

    triplets = set() #head, link, tail ---> process(spanID), operationName, process(references_spanID)
    
    dict_span_process = {}
    for elem in second:
        dict_span_process[elem['spanID']] = first['processes'][elem['processID']]['serviceName']

    root = first['traceID']

    for elem in second:
        head = elem['spanID']
        link = elem['operationName']
        tail = None
        if head != root:
            tail = elem['references'][0]['spanID']
            triplets.add(tuple([dict_span_process[head],link,dict_span_process[tail]]))
    
    return triplets
    


In [3]:
def create_inductive(triplets):
    df = pd.DataFrame(triplets,columns=['dm','rpctype','um'])

    df

    entities = set(df['dm']) | set(df['um'])
    relations = set(df['rpctype'])
    
    # check that there are enough rows to create two separate sets and having all the relations types in the train set
    
    assert len(df) > len(relations)

#     entities = set()
#     relations = set()
#     for i in range(len(df)):
#         head = df.iloc[i]['dm']
#         tail = df.iloc[i]['um']
#         rel = df.iloc[i]['rpctype']
#         entities.add(head)
#         entities.add(tail)
#         relations.add(rel)
#         triplets.append([head,rel,tail])



    # Create a graph object
    G = nx.Graph()

    for entity in entities:
        G.add_node(entity)
    for triplet in triplets:
        G.add_edge(triplet[0], triplet[2], label=triplet[1])

    fig = plt.figure(1, figsize=(20, 20), dpi=50)

    # Use NetworkX's built-in drawing function to visualize the graph
    nx.draw(G, node_size = 5)

    # Import matplotlib library to show the graph
    plt.show()



    # shuffle the nodes randomly
    nodes = list(G.nodes())
    random.shuffle(nodes)

    # split the nodes into two sets of approximately 64% and 36%
    n = len(nodes)
    m = int(n * 0.64)
    nodes1 = set(nodes[:m])
    nodes2 = set(nodes[m:])

    # create subgraphs from the two sets of nodes
    G1 = G.subgraph(nodes1)
    G2 = G.subgraph(nodes2)

    # print the sizes of the subgraphs
    print(len(G1), len(G2))

    # Use NetworkX's built-in drawing function to visualize the graph
    nx.draw(G1, node_size = 5)

    # Import matplotlib library to show the graph
    plt.show()

    # Use NetworkX's built-in drawing function to visualize the graph
    nx.draw(G2, node_size = 5)

    # Import matplotlib library to show the graph
    plt.show()

    len(list(G1.edges(data=True)))

    len(list(G2.edges(data=True)))

    entities = list(set(df['dm']) | set(df['um']))
    len(entities)

    relations = set(df['rpctype'])
    len(relations)

    G1_list = list(G1.nodes())
    G2_list = list(G2.nodes())

    #Check that there are no overlapping nodes

    assert len(set(G1_list) & set(G2_list)) == 0

    transductive = df[(df['um'].isin(G1_list)) & (df['dm'].isin(G1_list))]
    transductive

    inductive = df[(df['um'].isin(G2_list)) & (df['dm'].isin(G2_list))]
    inductive

    #Check that the remaining values are correct

    assert len(transductive) == len(pd.merge(transductive,df))

    assert len(inductive) == len(pd.merge(inductive,df))

    train_df = transductive
    train_df

    len(set(inductive['dm']) | set(inductive['um']))

    inductive

    validation_df = inductive.sample(frac=0.44, random_state=seed)  # random sample
    validation_df

    test_df = inductive.drop(validation_df.index)  # drop the sampled rows to get the second DataFram
    test_df

    len(set(train_df['dm']) | set(train_df['um']))

    len(set(validation_df['dm']) | set(validation_df['um']))

    len(set(test_df['dm']) | set(test_df['um']))

    #Check to see if there is test leakage

    assert(len(pd.merge(train_df, validation_df))==0)

    assert(len(pd.merge(test_df, validation_df))==0)

    assert(len(pd.merge(train_df, test_df))==0)

    assert(len(pd.merge(train_df, inductive))==0)

    #Check that all relations are present in the train set

    assert(len(set(df['rpctype'])) == len(set(train_df['rpctype'])))

    #Check that there is no entities test leakage

    train_entities = set(train_df['dm']) | set(train_df['um'])
    assert(len(train_entities & (set(validation_df['dm']) | set(validation_df['um'])))==0)

    assert(len(train_entities & (set(inductive['dm']) | set(inductive['um'])))==0)

    assert(len(train_entities & (set(test_df['dm']) | set(test_df['um'])))==0)
    return train_df, validation_df, test_df

In [4]:
train_df, validation_df, test_df = create_inductive(
#     create_triplets('social-network/sample-rate-100/user-timeline-service.json'))
    create_triplets('social-network/sample-rate-100/user-service.json'))

AssertionError: 

In [ ]:
train_df

In [ ]:
validation_df

In [ ]:
test_df

In [5]:
train_df, validation_df, test_df = create_inductive(
     create_triplets('social-network/sample-rate-100/user-timeline-service.json') |
    create_triplets('social-network/sample-rate-100/social-graph-service.json')|
    create_triplets('social-network/sample-rate-100/nginx-web-server.json')|
    create_triplets('social-network/sample-rate-100/post-storage-service.json')|
     create_triplets('social-network/sample-rate-100/user-service.json') )

AssertionError: 